In [9]:
!pip install streamlit pyjwt bcrypt python-dotenv pyngrok nltk streamlit-option-menu plotly textstat PyPDF2 -q

In [16]:
%%writefile db.py
import sqlite3
import bcrypt
import datetime
import time

DB_NAME = "users.db"

# ---------- SECURITY CONFIG ----------
MAX_LOGIN_ATTEMPTS = 3
LOCKOUT_TIME = 60  # seconds (sir changed from 5 min → 1 min)


# ================= DATABASE INIT =================
def init_db():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    # --- Users table ---
    c.execute("""
        CREATE TABLE IF NOT EXISTS users (
            email TEXT PRIMARY KEY,
            password BLOB NOT NULL,
            created_at TEXT NOT NULL,
            security_q1 TEXT NOT NULL,
            security_q2 TEXT NOT NULL,
            security_q3 TEXT NOT NULL
        )
    """)

    # --- Password history ---
    c.execute("""
        CREATE TABLE IF NOT EXISTS password_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            email TEXT,
            password BLOB,
            set_at TEXT,
            FOREIGN KEY(email) REFERENCES users(email)
        )
    """)

    # --- Login attempts (rate limiting) ---
    c.execute("""
        CREATE TABLE IF NOT EXISTS login_attempts (
            email TEXT PRIMARY KEY,
            attempts INTEGER DEFAULT 0,
            last_attempt REAL
        )
    """)

    conn.commit()
    conn.close()

    # Ensure admin exists
    init_admin()


# ================= ADMIN INIT =================
def init_admin():
    admin_email = "admin@llm.com"
    admin_password = "Admin@123"

    if not check_user_exists(admin_email):
        # default security answers for admin
        q1 = "adminpet"
        q2 = "adminmother"
        q3 = "adminteacher"

        register_user(admin_email, admin_password, q1, q2, q3)
        print("✅ Admin account created.")


# ================= HELPERS =================
def _timestamp():
    return datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")


# ================= RATE LIMIT =================
def _record_failed_attempt(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    now = time.time()
    c.execute("SELECT attempts, last_attempt FROM login_attempts WHERE email = ?", (email,))
    row = c.fetchone()

    if row:
        attempts, last = row

        # reset window if lock expired
        if now - last > LOCKOUT_TIME:
            c.execute(
                "UPDATE login_attempts SET attempts = 1, last_attempt = ? WHERE email = ?",
                (now, email),
            )
        else:
            c.execute(
                "UPDATE login_attempts SET attempts = ?, last_attempt = ? WHERE email = ?",
                (attempts + 1, now, email),
            )
    else:
        c.execute(
            "INSERT INTO login_attempts (email, attempts, last_attempt) VALUES (?, 1, ?)",
            (email, now),
        )

    conn.commit()
    conn.close()


def _reset_attempts(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("DELETE FROM login_attempts WHERE email = ?", (email,))
    conn.commit()
    conn.close()


def is_rate_limited(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT attempts, last_attempt FROM login_attempts WHERE email = ?", (email,))
    row = c.fetchone()
    conn.close()

    if row:
        attempts, last = row
        remaining = LOCKOUT_TIME - (time.time() - last)

        if attempts >= MAX_LOGIN_ATTEMPTS and remaining > 0:
            return True, int(remaining)

    return False, 0


# ================= USER MANAGEMENT =================
def register_user(email, password, q1, q2, q3):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    try:
        hashed = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
        now = _timestamp()

        c.execute(
            "INSERT INTO users (email, password, created_at, security_q1, security_q2, security_q3) VALUES (?, ?, ?, ?, ?, ?)",
            (email, hashed, now, q1, q2, q3)
        )

        c.execute(
            "INSERT INTO password_history (email, password, set_at) VALUES (?, ?, ?)",
            (email, hashed, now)
        )

        conn.commit()
        return True

    except sqlite3.IntegrityError:
        return False

    finally:
        conn.close()


def authenticate_user(email, password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password FROM users WHERE email = ?", (email,))
    data = c.fetchone()
    conn.close()

    if data and bcrypt.checkpw(password.encode(), data[0]):
        _reset_attempts(email)
        return True

    _record_failed_attempt(email)
    return False


def check_user_exists(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT 1 FROM users WHERE email = ?", (email,))
    exists = c.fetchone() is not None
    conn.close()
    return exists


# ================= PASSWORD SECURITY =================
def check_password_reused(email, new_password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password FROM password_history WHERE email = ?", (email,))
    history = c.fetchall()
    conn.close()

    for (stored_hash,) in history:
        if bcrypt.checkpw(new_password.encode(), stored_hash):
            return True

    return False


def update_password(email, new_password):
    hashed = bcrypt.hashpw(new_password.encode(), bcrypt.gensalt())
    now = _timestamp()

    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    c.execute("UPDATE users SET password = ? WHERE email = ?", (hashed, email))
    c.execute(
        "INSERT INTO password_history (email, password, set_at) VALUES (?, ?, ?)",
        (email, hashed, now),
    )

    conn.commit()
    conn.close()


# ================= ADMIN FUNCTIONS =================
def get_all_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT email, created_at FROM users ORDER BY created_at DESC")
    users = c.fetchall()
    conn.close()
    return users


def delete_user(email):
    # Protect admin account
    if email == "admin@llm.com":
        return False

    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    c.execute("DELETE FROM users WHERE email = ?", (email,))
    c.execute("DELETE FROM password_history WHERE email = ?", (email,))
    c.execute("DELETE FROM login_attempts WHERE email = ?", (email,))

    conn.commit()
    conn.close()
    return True


Overwriting db.py
Overwriting db.py


In [17]:
%%writefile readability.py
import textstat

class ReadabilityAnalyzer:
    def __init__(self, text):
        self.text = text
        self.num_sentences = textstat.sentence_count(text)
        self.num_words = textstat.lexicon_count(text, removepunct=True)
        self.num_syllables = textstat.syllable_count(text)
        self.complex_words = textstat.difficult_words(text)
        self.char_count = textstat.char_count(text)

    def get_all_metrics(self):
        return {
            "Flesch Reading Ease": textstat.flesch_reading_ease(self.text),
            "Flesch-Kincaid Grade": textstat.flesch_kincaid_grade(self.text),
            "SMOG Index": textstat.smog_index(self.text),
            "Gunning Fog": textstat.gunning_fog(self.text),
            "Coleman-Liau": textstat.coleman_liau_index(self.text)
        }


Overwriting readability.py
Overwriting readability.py


In [18]:
%%writefile auth.py
import jwt
import datetime
import os

# Get secret from environment (Colab Secret → env variable)
SECRET = os.getenv("JWT_SECRET_KEY", "dev_secret")


def create_token(email):
    payload = {
        "sub": email,
        "exp": datetime.datetime.utcnow() + datetime.timedelta(hours=2)
    }

    token = jwt.encode(payload, SECRET, algorithm="HS256")
    return token


Overwriting auth.py
Overwriting auth.py


In [19]:
from google.colab import userdata
import os

print("🔐 Checking Colab Secrets...\n")

secrets = [
    "JWT_SECRET_KEY",
    "EMAIL_ID",
    "EMAIL_APP_PASSWORD",
    "ADMIN_EMAIL_ID",
    "NGROK_AUTHTOKEN"
]

for key in secrets:
    try:
        value = userdata.get(key)

        if value:
            os.environ[key] = value
            print(f"✅ {key} loaded")
        else:
            print(f"❌ {key} is EMPTY")

    except Exception as e:
        print(f"⚠️ {key} missing → {e}")


🔐 Checking Colab Secrets...

✅ JWT_SECRET_KEY loaded
✅ EMAIL_ID loaded
✅ EMAIL_APP_PASSWORD loaded
✅ ADMIN_EMAIL_ID loaded
✅ NGROK_AUTHTOKEN loaded
🔐 Checking Colab Secrets...

✅ JWT_SECRET_KEY loaded
✅ EMAIL_ID loaded
✅ EMAIL_APP_PASSWORD loaded
✅ ADMIN_EMAIL_ID loaded
✅ NGROK_AUTHTOKEN loaded


In [20]:
%%writefile app.py
import streamlit as st
import re
import os
import random
import smtplib
from email.mime.text import MIMEText
import readability
import PyPDF2
import plotly.graph_objects as go

import db
from auth import create_token

# ---------- INIT DB ----------
db.init_db()

# ---------- LOAD ENV SECRETS ----------
EMAIL_ID = os.getenv("EMAIL_ID")
EMAIL_PASS = os.getenv("EMAIL_APP_PASSWORD")
ADMIN_EMAIL = os.getenv("ADMIN_EMAIL_ID")
ADMIN_PASS = os.getenv("ADMIN_PASSWORD", "Admin@123")  # fallback safe

# ---------- PAGE CONFIG ----------
st.set_page_config(page_title="PolicyNav Secure Portal", layout="wide")

# ---------- CUSTOM CSS ----------
st.markdown("""
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');

.stApp {
    background: #f6f3ee;
    font-family: 'Inter', sans-serif;
}

/* Title */
.portal-title {
    font-size: 2.6rem;
    font-weight: 700;
    text-align: center;
    color: #2f2620;
    margin-top: 2rem;
}

.portal-subtitle {
    text-align: center;
    color: #7a6a58;
    margin-bottom: 3rem;
}

/* Role Buttons */
.role-container {
    display: flex;
    justify-content: center;
    gap: 30px;
    margin-bottom: 2rem;
}

.role-card {
    width: 220px;
    padding: 2rem;
    border-radius: 18px;
    background: #ffffff;
    box-shadow: 0 10px 30px rgba(0,0,0,0.05);
    border: 1px solid #ece6dd;
    text-align: center;
    cursor: pointer;
    transition: 0.2s ease;
}

.role-card:hover {
    transform: translateY(-5px);
    box-shadow: 0 12px 30px rgba(120,90,60,0.2);
}

/* Form Card */
.form-card {
    width: 420px;
    margin: auto;
    background: #ffffff;
    padding: 2rem 2.5rem;
    border-radius: 20px;
    box-shadow: 0 10px 30px rgba(0,0,0,0.05);
    border: 1px solid #ece6dd;
}

/* Inputs */
.stTextInput input {
    border-radius: 12px;
    border: 1px solid #e7dccf;
    padding: 0.6rem;
    background: #fffdf9;
}

.stTextInput input:focus {
    border-color: #b08968;
    box-shadow: 0 0 0 2px rgba(176,137,104,0.15);
}

/* Buttons */
.stButton > button {
    width: 100%;
    border-radius: 12px;
    padding: 0.65rem 1rem;
    background: linear-gradient(135deg, #b08968, #9c6644);
    color: white;
    border: none;
    font-weight: 600;
    transition: 0.2s ease;
}

.stButton > button:hover {
    background: linear-gradient(135deg, #9c6644, #7f5539);
    transform: translateY(-1px);
    box-shadow: 0 8px 20px rgba(120,90,60,0.25);
}

#MainMenu {visibility: hidden;}
footer {visibility: hidden;}
section[data-testid="stSidebar"] {display: none;}
</style>
""", unsafe_allow_html=True)

# ---------- PAGE CONFIG ----------
st.set_page_config(page_title="PolicyNav Secure Portal", layout="wide")

# ---------- SESSION ----------
if "role" not in st.session_state:
    st.session_state.role = None

# ---------- MAIN LANDING PAGE ----------
if st.session_state.role is None:

    st.markdown("""
    <div class="portal-title">PolicyNav Secure Portal</div>
    <div class="portal-subtitle">
        AI-Powered Policy Intelligence & Secure Authentication
    </div>
    """, unsafe_allow_html=True)

    col1, col2 = st.columns(2)

    with col1:
        if st.button("🔐 User Portal", key="user_portal"):
            st.session_state.role = "user"
            st.rerun()

    with col2:
        if st.button("🛡 Admin Portal", key="admin_portal"):
            st.session_state.role = "admin"
            st.rerun()


# ---------- USER PORTAL ----------
elif st.session_state.role == "user":

    st.markdown("""
    <div class="portal-title">User Portal</div>
    """, unsafe_allow_html=True)

    st.markdown('<div class="form-card">', unsafe_allow_html=True)

    tab1, tab2 = st.tabs(["Login", "Sign Up"])

    # ---- LOGIN TAB ----
    with tab1:
        st.subheader("User Login")
        email = st.text_input("Email", key="user_email")
        password = st.text_input("Password", type="password", key="user_pass")

        if st.button("Login", key="user_login_btn"):
            st.success("User login logic here")

    # ---- SIGNUP TAB ----
    with tab2:
        st.subheader("Create Account")
        new_email = st.text_input("New Email", key="signup_email")
        new_pass = st.text_input("New Password", type="password", key="signup_pass")

        if st.button("Register", key="signup_btn"):
            st.success("Signup logic here")

    if st.button("⬅ Back to Home", key="user_back"):
        st.session_state.role = None
        st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)


# ---------- ADMIN PORTAL ----------
elif st.session_state.role == "admin":

    st.markdown("""
    <div class="portal-title">Admin Portal</div>
    """, unsafe_allow_html=True)

    st.markdown('<div class="form-card">', unsafe_allow_html=True)

    st.subheader("Admin Login")

    admin_email = st.text_input("Admin Email", key="admin_email")
    admin_pass = st.text_input("Admin Password", type="password", key="admin_pass")

    if st.button("Login as Admin", key="admin_login_btn"):
        st.success("Admin login logic here")

    if st.button("⬅ Back to Home", key="admin_back"):
        st.session_state.role = None
        st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)

# ---------- ADMIN SECTION ----------
elif st.session_state.role == "admin":

    st.markdown('<div class="form-card">', unsafe_allow_html=True)

    st.subheader("Admin Login")

    admin_email = st.text_input("Admin Email")
    admin_pass = st.text_input("Admin Password", type="password")

    if st.button("Login as Admin"):
        st.success("Admin login logic here")

    if st.button("⬅ Back"):
        st.session_state.role = None
        st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)


# ---------- SESSION STATE ----------
defaults = {
    "page": "Login",
    "otp": None,
    "otp_email": None,
    "logged_in": False,
    "admin": False,
    "token": None,
}

for k, v in defaults.items():
    if k not in st.session_state:
        st.session_state[k] = v


def switch(page):
    st.session_state.page = page
    st.rerun()
# ---------- OTP EMAIL ----------
def send_otp(email):
    import time

    otp = str(random.randint(100000, 999999))
    st.session_state.otp = otp
    st.session_state.otp_email = email

    # DEBUG: show target email
    st.write("📨 Sending OTP to:", email)

    if not EMAIL_ID or not EMAIL_PASS:
        st.error("❌ Email credentials missing in environment.")
        return False

    msg = MIMEText(f"""
Hello,

Your PolicyNav OTP is: {otp}

This OTP is valid for login verification.

If you did not request this, please ignore.
""")

    # 🔹 change subject every time to avoid Gmail blocking
    msg["Subject"] = f"PolicyNav OTP {random.randint(1000,9999)}"
    msg["From"] = EMAIL_ID
    msg["To"] = email

    try:
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)

        # 🔹 small delay avoids Gmail anti-bot
        time.sleep(2)

        server.login(EMAIL_ID, EMAIL_PASS)
        server.send_message(msg)
        server.quit()

        st.success("✅ OTP email sent successfully (SMTP confirmed)")
        return True

    except Exception as e:
        st.error(f"❌ OTP Email Error: {e}")
        return False

# ================= SIGNUP =================
if st.session_state.page == "Signup":

    st.title("Create Account")

    email = st.text_input("Email")
    password = st.text_input("Password", type="password")

    st.markdown("### 🔐 Security Questions")

    q1 = st.text_input("1️⃣ What is your pet name?")
    q2 = st.text_input("2️⃣ What is your mother's maiden name?")
    q3 = st.text_input("3️⃣ Who is your favorite teacher?")

    if st.button("Register"):

        # Email validation
        if not re.match(r"[^@]+@[^@]+\.[a-zA-Z]{2,}", email):
            st.error("Invalid email format")

        # Password validation
        elif not password.isalnum():
            st.error("Password must be alphanumeric")

        # Security answers validation
        elif not q1 or not q2 or not q3:
            st.error("All security answers are required")

        elif " " in q1 or " " in q2 or " " in q3:
            st.error("Security answers cannot contain spaces")

        elif db.check_user_exists(email):
            st.error("User already exists")

        else:
            db.register_user(email, password, q1.strip(), q2.strip(), q3.strip())
            st.success("Registration successful")
            switch("Login")

    if st.button("Back to Login"):
        switch("Login")


# ================= LOGIN =================
elif st.session_state.page == "Login" and not st.session_state.logged_in:

    st.title("User Login")
    st.markdown('<div class="split-layout"><div class="left-panel">', unsafe_allow_html=True)

    email = st.text_input("Email")
    password = st.text_input("Password", type="password")

    if st.button("Login"):

        # 🔒 Check lock
        locked, wait = db.is_rate_limited(email)
        if locked:
            st.error(f"Account locked. Try again in {wait} seconds.")
            st.stop()

        # 🔑 Authenticate
        if db.authenticate_user(email, password):

            # Send OTP
            if send_otp(email):
                st.success("OTP sent to registered email")
                switch("OTP")
            else:
                st.error("Failed to send OTP. Check email configuration.")

        else:
            st.error("Invalid email or password")

    st.write("---")

    if st.button("Signup"):
        switch("Signup")

    if st.button("Forgot Password"):
        switch("Forgot")

    # ---------- ADMIN LOGIN ----------
    st.markdown('<div class="section-divider"></div>', unsafe_allow_html=True)
    st.markdown('<div class="section-title">Admin Login</div>', unsafe_allow_html=True)

    a_email = st.text_input("Admin Email")
    a_pass = st.text_input("Admin Password", type="password")

    if st.button("Admin Login"):

        if a_email == ADMIN_EMAIL and a_pass == ADMIN_PASS:
            st.session_state.admin = True
            st.session_state.logged_in = True
            st.session_state.token = create_token(a_email)
            switch("Admin")
        else:
            st.error("Invalid admin credentials")


# ================= OTP VERIFY =================
elif st.session_state.page == "OTP":

    st.title("OTP Verification")

    user_otp = st.text_input("Enter OTP")

    if st.button("Verify OTP"):

        if user_otp == st.session_state.otp:
            st.session_state.logged_in = True
            st.session_state.token = create_token(st.session_state.otp_email)
            st.success("Login successful")
            switch("Dashboard")
        else:
            st.error("Invalid OTP")


# ================= FORGOT PASSWORD =================
elif st.session_state.page == "Forgot":

    st.title("Reset Password")

    email = st.text_input("Registered Email")
    new_pass = st.text_input("New Password", type="password")

    if st.button("Reset Password"):

        if not db.check_user_exists(email):
            st.error("User not found")

        elif not new_pass.isalnum():
            st.error("Password must be alphanumeric")

        elif db.check_password_reused(email, new_pass):
            st.error("Cannot reuse old password")

        else:
            db.update_password(email, new_pass)
            st.success("Password updated successfully")
            switch("Login")

    if st.button("Back"):
        switch("Login")

# ================= READABILITY PAGE =================
elif st.session_state.page == "Readability" and st.session_state.logged_in:

    st.title("📖 Text Readability Analyzer")

    tab1, tab2 = st.tabs(["✍️ Input Text", "📂 Upload File"])

    text_input = ""

    # ---------- TEXT INPUT ----------
    with tab1:
        raw_text = st.text_area("Enter text to analyze (min 50 characters):", height=200)
        if raw_text:
            text_input = raw_text

    # ---------- FILE UPLOAD ----------
    with tab2:
        uploaded_file = st.file_uploader("Upload TXT or PDF", type=["txt", "pdf"])

        if uploaded_file:
            try:
                if uploaded_file.type == "application/pdf":
                    reader = PyPDF2.PdfReader(uploaded_file)
                    text = ""
                    for page in reader.pages:
                        text += page.extract_text() + "\n"
                    text_input = text
                    st.success(f"Loaded {len(reader.pages)} pages from PDF.")
                else:
                    text_input = uploaded_file.read().decode("utf-8")
                    st.success("TXT file loaded successfully.")
            except Exception as e:
                st.error(f"Error reading file: {e}")

    # ---------- ANALYZE ----------
    if st.button("Analyze Readability"):

        if len(text_input) < 50:
            st.error("Text too short. Minimum 50 characters required.")
        else:
            analyzer = readability.ReadabilityAnalyzer(text_input)
            scores = analyzer.get_all_metrics()

            st.markdown("---")
            st.subheader("📊 Readability Results")

            def gauge(value, title, max_val):
                fig = go.Figure(go.Indicator(
                    mode="gauge+number",
                    value=value,
                    title={'text': title},
                    gauge={'axis': {'range': [0, max_val]}}
                ))
                st.plotly_chart(fig, use_container_width=True)

            col1, col2, col3 = st.columns(3)

            with col1:
                gauge(scores["Flesch Reading Ease"], "Flesch Ease", 100)

            with col2:
                gauge(scores["Flesch-Kincaid Grade"], "Kincaid Grade", 20)

            with col3:
                gauge(scores["SMOG Index"], "SMOG Index", 20)

            col4, col5 = st.columns(2)

            with col4:
                gauge(scores["Gunning Fog"], "Gunning Fog", 20)

            with col5:
                gauge(scores["Coleman-Liau"], "Coleman-Liau", 20)

            st.markdown("### 📝 Text Statistics")

            s1, s2, s3, s4, s5 = st.columns(5)
            s1.metric("Sentences", analyzer.num_sentences)
            s2.metric("Words", analyzer.num_words)
            s3.metric("Syllables", analyzer.num_syllables)
            s4.metric("Complex Words", analyzer.complex_words)
            s5.metric("Characters", analyzer.char_count)

    if st.button("⬅ Back to Dashboard"):
        switch("Dashboard")

# ================= USER DASHBOARD ================
elif st.session_state.page == "Dashboard" and st.session_state.logged_in:

    st.title("User Dashboard")
    st.success("JWT Authenticated Session Active")

    if st.button("📖 Open Readability Analyzer"):
        switch("Readability")

    if st.button("Logout"):
        st.session_state.clear()
        switch("Login")


# ================= ADMIN DASHBOARD =================
elif st.session_state.page == "Admin" and st.session_state.admin:

    st.title("Admin Dashboard")

    users = db.get_all_users()
    st.write("Registered Users:")
    st.table(users)

    email_del = st.text_input("Delete user email")

    if st.button("Delete User"):
        if db.delete_user(email_del):
            st.success("User deleted")
        else:
            st.error("Cannot delete admin")

    if st.button("Logout Admin"):
        st.session_state.clear()
        switch("Login")


Overwriting app.py
Overwriting app.py


In [21]:
!ngrok config add-authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [22]:
# ---------- FINAL RUN CELL ----------
import os, subprocess, time
from google.colab import userdata
from pyngrok import ngrok

print("🔐 Loading secrets...")

JWT_SECRET = userdata.get("JWT_SECRET_KEY")
EMAIL_PASS = userdata.get("EMAIL_APP_PASSWORD")
EMAIL_ID = userdata.get("EMAIL_ID")
NGROK_TOKEN = userdata.get("NGROK_AUTHTOKEN")

if JWT_SECRET:
    os.environ["JWT_SECRET_KEY"] = JWT_SECRET
if EMAIL_PASS:
    os.environ["EMAIL_APP_PASSWORD"] = EMAIL_PASS
if EMAIL_ID:
    os.environ["EMAIL_ID"] = EMAIL_ID

if not NGROK_TOKEN:
    print("❌ NGROK token missing")
else:
    ngrok.set_auth_token(NGROK_TOKEN)
    ngrok.kill()
    os.system("pkill -f streamlit")
    time.sleep(2)

    process = subprocess.Popen(["streamlit", "run", "app.py"], env=os.environ.copy())
    time.sleep(6)

    public_url = ngrok.connect(8501).public_url
    print("🌍 OPEN:", public_url)

    input("Press ENTER to stop...")
    process.terminate()
    ngrok.kill()


🔐 Loading secrets...
🌍 OPEN: https://justine-strikebound-independently.ngrok-free.dev
Press ENTER to stop...
